## Exercise 1

Åbn 'https://www.dba.dk/biler/'.
Find ind alle biler med mærket: 'Peugeot' og gem dem.

In [2]:
from tqdm import tqdm
import bs4
import requests
import re
    
def scrape_for_cars(url, search_year_flag, year_to_filter_by):
    """
    returns:
        A list of tuples of strings with information about cars of the type Peugot
    """
    
    r = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'})
    r.raise_for_status()

    soup = bs4.BeautifulSoup(r.text, 'html.parser')
    event_cells = soup.select("tr[class^=dbaListing]")
    
    scraped_cars_per_page = []
    
    print(len(event_cells))
    
    for event_cell in event_cells:
        
        to_find = re.compile(r'title="Modelår">[\r\n]+([^\r\n]+)')
        regex_car_year = to_find.search(str(event_cell))
        
        car_year = str(event_cell.select("td[title^=Modelår]")[0].getText()).strip()
        car_km = event_cell.select("td[title^=Km]")[0].getText().strip()
        car_km = car_km.replace(".", "")
        car_price = event_cell.select("td[title^=Pris]")[0].getText().strip()     
        car_price = car_price[:-4]
        car_price = car_price.replace(".","")
        
        for link in event_cell.find_all('a'):
            if not link.get('href').startswith('https'):
                continue
            car_link = link.get('href')
        
        if(search_year_flag):       
            if(str(regex_car_year.group(1)).strip() == str(year_to_filter_by)):
                scraped_cars_per_page.append((car_link, car_year, car_km, car_price))
        else:
            scraped_cars_per_page.append((car_link, car_year, car_km, car_price))
    
    print(scraped_cars_per_page)
              
    return scraped_cars_per_page


scraped_total_cars = []

indexes = range(1, 6)
search_year_flag = True
year_to_filter_by = 2009

for idx in tqdm(indexes):
    if idx == 0:
        url="https://www.dba.dk/biler/biler/maerke-peugeot/"
    else:
        url = "https://www.dba.dk/biler/biler/maerke-peugeot/side-{}/".format(idx)
        
    scraped_total_cars += scrape_for_cars(url, search_year_flag, year_to_filter_by)
    
print(len(scraped_total_cars))

 20%|██        | 1/5 [00:00<00:03,  1.18it/s]

41
[('https://www.dba.dk/peugeot-308-diesel-2009-km/id-1078397400/', '2009', '198000', '33000'), ('https://www.dba.dk/peugeot-308-16-hdi-109/id-505603203/', '2009', '320000', '29800'), ('https://www.dba.dk/peugeot-206-14-hdi-comfort/id-1078621466/', '2009', '140000', '29999')]


 40%|████      | 2/5 [00:01<00:02,  1.17it/s]

21
[('https://www.dba.dk/peugeot-207-16-vti-comfort/id-505595183/', '2009', '156000', '44900'), ('https://www.dba.dk/peugeot-308-20-hdi-136/id-505489630/', '2009', '230000', '39900'), ('https://www.dba.dk/peugeot-207-16-cc-benzin/id-505567226/', '2009', '120000', '84900')]


 60%|██████    | 3/5 [00:02<00:01,  1.17it/s]

21
[('https://www.dba.dk/peugeot-207-16-hdi-xr-diesel/id-505116368/', '2009', '190000', '24999')]


 80%|████████  | 4/5 [00:03<00:00,  1.12it/s]

21
[('https://www.dba.dk/peugeot-206-14-hdi-70/id-505602747/', '2009', '176000', '24900'), ('https://www.dba.dk/peugeot-3008-16-hdi-110/id-505602506/', '2009', '272000', '34900')]


100%|██████████| 5/5 [00:04<00:00,  1.13it/s]

21
[('https://www.dba.dk/peugeot-207-16-hdi-90/id-505587500/', '2009', '193000', '21000')]
10


## Exercise 2
Brug Regular Expressions til at sortere de fundne biler på år 2009.

In [ ]:
#Der er implementeret en løsning hvor den leder efter year_to_filter_by hvis search_year_flag er sat til True

## Exercise 3
Brug Selenium til at tilgå de samme punkter som i forrige opgaver, og vælge den dyreste bil.
Brug BeautifulSoup til at printe bilbeskrivelsen ud.

In [1]:
from selenium import webdriver
import bs4
import json
from webdriver_manager.firefox import GeckoDriverManager

from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

url = 'https://www.dba.dk/biler/biler/maerke-peugeot/'
def get_cars_selenium():
    
    browser = webdriver.Firefox(executable_path=GeckoDriverManager().install())
    browser.get(url)
    browser.implicitly_wait(2)
    button = browser.find_element_by_id('onetrust-accept-btn-handler')
    button.click()
    
    all_cars = []
    
    number_of_pages = browser.find_element_by_css_selector('.pagination > ul:nth-child(1) > li:nth-child(6) > a:nth-child(1)')
    number_of_pages = int(str(number_of_pages.text).strip())
    
    for n in range(1,number_of_pages-90):
        if(n != 1):
            browser.get(url + 'side-' + str(n))
            
        
        soup = bs4.BeautifulSoup(requests.get(browser.current_url, headers={'User-Agent': 'Mozilla/5.0'}).text, 'html.parser')
        cell_count = soup.select("tr[class^=dbaListing]")
        print('fetching from ' + str(browser.current_url) + ', cell count:' + str(len(cell_count)))
        browser.implicitly_wait(2)
        browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        
        car_single_page = []
        
        for num in range(4, len(cell_count)+5):
            try:
                km = str(browser.find_element_by_css_selector('tr.dbaListing:nth-child(' + str(num) +') > td:nth-child(3)').text)
                km = km.replace(".","")
                year = str(browser.find_element_by_css_selector('tr.dbaListing:nth-child(' + str(num) +') > td:nth-child(4)').text)
                price = str(browser.find_element_by_css_selector('tr.dbaListing:nth-child(' + str(num) +') > td:nth-child(6) > a:nth-child(1)').text)
                price = price.replace(".","")
                car_single_page.append((year, km, price))
            except:
                pass
        #print(car_single_page)
        all_cars += car_single_page
    return all_cars

total = get_cars_selenium()
print(len(total))
print(total)

ModuleNotFoundError: No module named 'webdriver_manager'